In [2]:
import numpy as np
import pandas as pd

import xgboost as xgb
import lightgbm as lgb
import catboost as cat

from sklearn import preprocessing
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_squared_error

In [3]:
# For training the model
X_train_realmean = pd.read_csv("../X/Xtrainmean.csv")

# For cross validation
X_valid_realmean = pd.read_csv("../X/Xvalidmean.csv")

# For prediction
X_test_realmean = pd.read_csv("../X/Xtestmean.csv")

In [5]:
y_train_t1 = pd.read_csv("../Task1/Y_train.csv")
y_valid_t1 = pd.read_csv("../Task1/Y_valid.csv")

In [6]:
y_train_t2_value=y_train_t1["mort_icu"]
y_valid_t2_value=y_valid_t1["mort_icu"]

In [19]:
test=X_test_realmean

# Eliminate feature containing 70% 0 value
import copy
X_train_nozero=copy.deepcopy(X_train_realmean)
X_valid_nozero=copy.deepcopy(X_valid_realmean)
X_test_nozero=copy.deepcopy(X_test_realmean)
for i in X_train_realmean.columns:
    if (X_train_nozero[i] == 0).sum()> 12000:
        X_train_nozero.drop(i, axis=1, inplace=True)

headnozero=list(X_train_nozero.columns.values)
X_valid_nozero = X_valid_nozero[X_train_nozero.columns]
X_test_nozero = X_test_nozero[X_train_nozero.columns]

# Impute the 0 with mean
imp = SimpleImputer(missing_values=0, strategy='mean')
X_train_nozero = pd.DataFrame(imp.fit_transform(X_train_nozero))
X_train_nozero.columns=headnozero
X_valid_nozero = pd.DataFrame(imp.fit_transform(X_valid_nozero))
X_valid_nozero.columns=headnozero
X_test_nozero = pd.DataFrame(imp.fit_transform(X_test_nozero))
X_test_nozero.columns=headnozero
X_train_nozero

In [11]:
train = pd.merge(X_train_realmean,y_train_t1,on = 'Unnamed: 0',how='inner')
valid = pd.merge(X_valid_realmean,y_valid_t1,on = 'Unnamed: 0',how='inner')

In [12]:
train

,Unnamed: 0,alanine aminotransferase,albumin,albumin ascites,albumin pleural,albumin urine,alkaline phosphate,anion gap,asparate aminotransferase,basophils,...,tidal volume spontaneous,total protein,total protein urine,troponin-i,troponin-t,venous pvo2,weight,white blood cell count,white blood cell count urine,mort_icu
0,3_145834_211552,-0.254460,-1.979855,0.0,0.0,0.0,-0.318615,0.931458,-0.226618,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,1.026004,0.634186,0.160067,0
1,6_107064_228232,-0.256599,-0.251806,0.0,0.0,0.0,-0.422405,1.723627,-0.254291,-0.742403,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.181589,0.000000,0
2,9_150750_220597,-0.269432,0.000000,0.0,0.0,0.0,-0.367050,-0.330641,-0.271686,2.097036,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.181589,0.000000,1
3,11_194540_229441,0.000000,0.000000,0.0,0.0,0.0,0.000000,-0.088963,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.139623,0.000000,0
4,12_112213_232669,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.240269,0.000000,0.000000,...,-1.333059,0.0,0.0,0.0,-0.317391,0.0,0.000000,-0.414731,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16755,99966_167228_252173,0.000000,1.044231,0.0,0.0,0.0,0.000000,-0.572320,0.000000,0.290121,...,0.000000,0.0,0.0,0.0,-0.361918,0.0,0.584864,-0.156720,0.000000,0
16756,99973_150202_275083,0.000000,0.000000,0.0,0.0,0.0,0.000000,-0.290362,0.000000,-0.677870,...,0.000000,0.0,0.0,0.0,-0.364145,0.0,0.973702,0.429243,0.000000,0
16757,99982_151454_221194,-0.260876,1.044231,0.0,0.0,0.0,-0.297857,-0.330641,-0.261407,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,-0.443803,-0.682067,0.000000,0
16758,99991_151118_226241,0.000000,0.000000,0.0,0.0,0.0,0.000000,-0.753579,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.782719,-0.633884,-0.303768,0


In [13]:
##查看相关性
train.corr().iloc[:,-1].sort_values(ascending=False)

mort_icu                                     1.000000
anion gap                                    0.210730
lactate                                      0.193637
lactic acid                                  0.183405
blood urea nitrogen                          0.143719
                                               ...   
co2                                         -0.092061
mean corpuscular hemoglobin concentration   -0.102825
albumin                                     -0.112213
bicarbonate                                 -0.112932
glascow coma scale total                    -0.221923
Name: mort_icu, Length: 105, dtype: float64

In [14]:
valid.corr().iloc[:,-1].sort_values(ascending=False)

mort_icu                    1.000000
lactate                     0.194417
lactic acid                 0.186412
anion gap                   0.174031
blood urea nitrogen         0.132405
                              ...   
co2                        -0.092509
bicarbonate                -0.112102
glascow coma scale total   -0.212527
creatinine body fluid            NaN
lymphocytes atypical csl         NaN
Name: mort_icu, Length: 105, dtype: float64

In [15]:
##查看方差
train.var().sort_values(ascending=False)

mean corpuscular hemoglobin                  0.974906
mean corpuscular volume                      0.958519
platelets                                    0.925568
red blood cell count                         0.892813
mean corpuscular hemoglobin concentration    0.861872
                                               ...   
albumin pleural                              0.002685
creatinine ascites                           0.001253
lymphocytes atypical csl                     0.000716
creatinine pleural                           0.000716
creatinine body fluid                        0.000708
Length: 105, dtype: float64

In [16]:
train.corr().iloc[:,-1].sort_values(ascending=False)

mort_icu                                     1.000000
anion gap                                    0.210730
lactate                                      0.193637
lactic acid                                  0.183405
blood urea nitrogen                          0.143719
                                               ...   
co2                                         -0.092061
mean corpuscular hemoglobin concentration   -0.102825
albumin                                     -0.112213
bicarbonate                                 -0.112932
glascow coma scale total                    -0.221923
Name: mort_icu, Length: 105, dtype: float64

In [17]:
train.shape

(16760, 106)

In [20]:
##对训练和验证集删除ID字段
train.drop(['Unnamed: 0'],axis = 1,inplace=True)
valid.drop(['Unnamed: 0'],axis = 1,inplace=True)
test.drop(['Unnamed: 0'],axis = 1,inplace=True)

In [21]:
valid.var().sort_values(ascending=False)

platelets                                    0.943340
mean corpuscular hemoglobin                  0.923437
blood urea nitrogen                          0.905915
mean corpuscular hemoglobin concentration    0.868764
mean corpuscular volume                      0.867709
                                               ...   
calcium urine                                0.000781
red blood cell count ascites                 0.000398
creatinine ascites                           0.000058
creatinine body fluid                        0.000000
lymphocytes atypical csl                     0.000000
Length: 105, dtype: float64

In [28]:
train_low_var = train.var()[train.var()<0.08].index[:-1]

In [23]:
test.shape

(4790, 104)

In [29]:
test.drop(train_low_var,axis = 1,inplace=True)

In [30]:
train.drop(train_low_var,axis = 1,inplace=True)
valid.drop(train_low_var,axis = 1,inplace=True)
low_corr = train.corr().iloc[:,-1][abs(train.corr().iloc[:,-1])<0.005].index

train_1 = train.copy()
valid_1 = valid.copy()
test_1=test.copy()

In [ ]:
bycorrelation=train_1.iloc[:,:-1].to_csv('cor_train.csv',index=False)

In [ ]:
newpd.to_csv('validnewpd.csv')

In [ ]:
validbycorrelation=valid_1.iloc[:,:-1].to_csv('cor_valid.csv',index=False)

In [ ]:
testbycorrelation=test_1.to_csv('cor_test.csv',index=False)

In [ ]:
testbycorrelation

In [36]:
##模型的建立
##先使用lightgbm
train_feature = train_1.loc[:,train_1.columns!='mort_icu']
train_label = train_1.loc[:,train_1.columns=='mort_icu']

valid_feature = valid_1.loc[:,valid_1.columns!='mort_icu']
valid_label = valid_1.loc[:,valid_1.columns=='mort_icu']

In [38]:
cat_model = cat.CatBoostClassifier(n_estimators=2000,verbose=0)

cat_model.fit(train_feature.values,train_label)


##预测

y_valid_score = cat_model.decision_function(valid_feature.values)
# compute ROC curve and AUC for each learning rate
fpr, tpr, thresholds = roc_curve(valid_feature,y_valid_score)
roc_auc = auc(fpr, tpr)
print(roc_auc)

AttributeError: 'CatBoostClassifier' object has no attribute 'decision_function'

In [40]:
pred = cat_model.predict_proba(test_1)
pd.DataFrame(pred).to_csv('cat.csv',index=False)

In [ ]:
lgb_model = cat.CatBoostRegressor(n_estimators=2000,verbose=0)

lgb_model.fit(X_train_selected_t2_norm,y_train_t2_value)


##预测
pred = lgb_model.predict(X_valid_selected_t2_norm)

rmse = np.sqrt(mean_squared_error(y_valid_t2_value,pred))
print(rmse)

In [41]:
##dnn
from tensorflow import keras
from keras import optimizers
from keras import layers,models
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.layers import Conv1D,Flatten,MaxPooling1D,Dense,Dropout,Input,Reshape,Activation
from keras.models import Sequential
from keras.models import Sequential,Model
from keras.models import load_model

In [42]:
def design_model():
    # design network
    inp=Input(shape=(61,))
    reshape=Reshape((61,1))(inp)
    conv1=Conv1D(32,3,padding='same')(reshape)
    print(conv1)
    l1=Activation('relu')(conv1)
    print(l1)
    l2 = Dropout(0.2)(l1)
    m2=MaxPooling1D(pool_size=2,padding='valid')(l2)
    print(m2)
    m3 = Flatten()(m2)
    print(m3)
    m4 = Dense(8,activation='relu')(m3)
    m5 = Dense(1,activation='linear')(m4)
    model=Model(inputs = inp,outputs = m5)
    model.summary() #打印出模型概况
    model.compile(loss=["mse"], optimizer='adam',metrics=['mse'])
    
    return model
cnn_model = design_model()

KerasTensor(type_spec=TensorSpec(shape=(None, 61, 32), dtype=tf.float32, name=None), name='conv1d/BiasAdd:0', description="created by layer 'conv1d'")
KerasTensor(type_spec=TensorSpec(shape=(None, 61, 32), dtype=tf.float32, name=None), name='activation/Relu:0', description="created by layer 'activation'")
KerasTensor(type_spec=TensorSpec(shape=(None, 30, 32), dtype=tf.float32, name=None), name='max_pooling1d/Squeeze:0', description="created by layer 'max_pooling1d'")
KerasTensor(type_spec=TensorSpec(shape=(None, 960), dtype=tf.float32, name=None), name='flatten/Reshape:0', description="created by layer 'flatten'")
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 61)]              0         
                                                                 
 reshape (Reshape)           (None, 61, 1)             0         
                                 

In [43]:
history1 = cnn_model.fit(train_feature, train_label, epochs=300, batch_size=1200,  validation_data=[valid_feature, valid_label],verbose=0, shuffle=True)

ValueError: in user code:

    File "D:\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "D:\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "D:\anaconda3\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "D:\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" is '

    ValueError: Input 0 of layer "model" is inc

In [ ]:
##预测
pred = cnn_model.predict(X_valid_selected_t2_norm)

rmse = np.sqrt(mean_squared_error(y_valid_t2_value,pred))
print(rmse)

In [ ]:
train_feature

In [44]:
##模型融合
##单独模型建模的函数
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
def get_models():
    """Generate a library of base learners."""
    
    knn = KNeighborsRegressor()
    gb = GradientBoostingRegressor(n_estimators = 100,random_state=922)
    rf = RandomForestRegressor(n_estimators = 20,random_state=922)
    adat = AdaBoostRegressor(n_estimators = 100,random_state=922)
    lgb_model = lgb.LGBMRegressor(n_estimators=2000)
    cat_model = cat.CatBoostRegressor(verbose=0,n_estimators=2000)
    models = {
              'knn': knn,
              'rf': rf,
              'gb': gb,
              'adat':adat,
              'lgb':lgb_model,
              'cat':cat_model
              }

    return models

In [45]:
def train_predict(model_list,xtrain,xtest,ytrain,ytest):
    """Fit models in list on training set and return preds"""
    ##七个模型对于测试集的预测结果
    P1 = np.zeros((ytest.shape[0], len(model_list)))
    P1 = pd.DataFrame(P1)

    print("Fitting models.")
    cols = list()
    for i, (name, m) in enumerate(model_list.items()):
        print("%s..." % name, end=" ", flush=False)
        m.fit(xtrain, ytrain)
        P1.iloc[:, i] = m.predict(xtest)
        cols.append(name)
        print("done")

    P1.columns = cols
    print("Done.\n")
    return P1


def score_models(y,P1):
    """Score model in prediction DF"""
    print("Scoring models.")
    zb = pd.DataFrame()
    for m in P1.columns:
        rmse = np.sqrt(mean_squared_error(y,P1.loc[:,m]))
        zb = pd.concat([zb,pd.DataFrame(np.array([rmse]).reshape(-1,1),columns = [m],\
                             index = ['rmse'])],axis = 1)

        # print("%-26s: %.3f" % (m, roc_score))
    print("Done.\n")
    return zb

In [47]:
import warnings
warnings.filterwarnings('ignore')
models = get_models()
P1 = train_predict(models,train_feature.values,valid_feature.values,train_label.values,valid_label.values)
zb = score_models(valid_label.values,P1)

Fitting models.
knn... done
rf... done
gb... done
adat... done
lgb... done
cat... done
Done.

Scoring models.
Done.



In [48]:
P1

,knn,rf,gb,adat,lgb,cat
0,0.0,0.10,0.108487,0.041768,0.115392,0.111629
1,0.0,0.15,0.093330,0.041768,0.347747,0.149019
2,0.0,0.05,0.006062,0.041768,-0.045078,-0.022567
3,0.0,0.05,0.028470,0.041768,0.124130,0.047172
4,0.0,0.05,-0.016206,0.041768,-0.039644,-0.018420
...,...,...,...,...,...,...
2389,0.0,0.00,0.014229,0.041768,0.011707,-0.000133
2390,0.0,0.05,0.004939,0.041768,-0.021299,-0.018572
2391,0.0,0.55,0.226737,0.142925,0.282306,0.346407
2392,0.2,0.10,0.063671,0.041768,0.029758,0.073779


In [49]:
zb

,knn,rf,gb,adat,lgb,cat
rmse,0.236622,0.228028,0.2189,0.235682,0.221743,0.216937


In [58]:
test_1

,alanine aminotransferase,albumin,alkaline phosphate,anion gap,asparate aminotransferase,basophils,bicarbonate,bilirubin,blood urea nitrogen,calcium,...,systemic vascular resistance,systolic blood pressure,temperature,tidal volume observed,tidal volume set,tidal volume spontaneous,troponin-t,weight,white blood cell count,white blood cell count urine
0,0.000000,0.000000,0.000000,-0.088963,0.000000,0.000000,1.019964,0.000000,-0.717612,-0.187666,...,0.000000,-0.396711,0.702213,1.214153,2.536928,-0.353795,0.000000,0.000000,0.387277,0.0
1,-0.261946,1.044231,-0.138712,-0.733439,-0.259826,-0.226141,0.877078,-0.322540,-0.339235,-0.119024,...,0.000000,-0.281789,0.056583,0.000000,0.000000,0.000000,0.000000,-0.105634,1.437970,0.0
2,-0.251251,0.612219,0.117304,-0.330641,-0.265360,-0.484272,0.591306,-0.420192,-0.509118,0.395792,...,0.000000,0.140199,-0.974249,0.000000,0.000000,0.000000,0.000000,-0.517139,-0.410068,0.0
3,0.000000,0.000000,0.000000,-0.813998,0.000000,0.000000,-0.158846,0.000000,-0.138463,0.000000,...,-0.403265,-0.018154,1.058462,0.883270,0.859888,-0.094990,0.000000,0.220140,-0.452033,0.0
4,-0.271571,0.000000,-0.173309,-0.813998,-0.267732,1.580775,-0.051682,-0.420192,-0.416455,-0.736803,...,0.000000,-1.269928,1.361948,0.000000,0.000000,0.000000,0.000000,0.000000,-1.020899,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4785,0.000000,0.000000,0.000000,-0.572320,0.000000,0.000000,1.662951,0.000000,-0.045800,0.601718,...,0.000000,-0.312894,-0.819947,-0.599413,-2.494192,-0.101985,0.000000,-0.884455,-0.340125,0.0
4786,0.000000,0.000000,0.000000,1.216102,0.000000,0.000000,-0.587504,0.000000,2.057667,0.533076,...,0.000000,1.527590,-0.115897,0.000000,0.000000,0.000000,-0.339655,-0.370414,-0.041704,0.0
4787,0.000000,0.000000,0.000000,-0.693159,0.000000,0.000000,-0.587504,0.000000,-0.717612,0.000000,...,0.000000,-0.142656,0.342592,-0.059030,0.021368,-0.076804,0.000000,0.109258,-0.338260,0.0
4788,0.000000,0.000000,0.000000,0.636073,0.000000,-0.742403,-0.158846,0.000000,0.278523,0.258507,...,0.000000,0.555985,-1.013160,0.000000,0.000000,0.000000,-0.308485,-0.511336,0.318889,0.0


In [51]:
train_feature

,alanine aminotransferase,albumin,alkaline phosphate,anion gap,asparate aminotransferase,basophils,bicarbonate,bilirubin,blood urea nitrogen,calcium,...,systemic vascular resistance,systolic blood pressure,temperature,tidal volume observed,tidal volume set,tidal volume spontaneous,troponin-t,weight,white blood cell count,white blood cell count urine
0,-0.254460,-1.979855,-0.318615,0.931458,-0.226618,0.000000,-1.694872,-0.298127,0.792807,-1.004507,...,0.615623,-0.638602,-0.002273,0.800876,0.440628,0.000000,0.000000,1.026004,0.634186,0.160067
1,-0.256599,-0.251806,-0.422405,1.723627,-0.254291,-0.742403,-1.444821,-0.444605,1.737977,-0.016061,...,0.000000,1.537514,-0.518339,0.000000,0.000000,0.000000,0.000000,0.000000,-0.181589,0.000000
2,-0.269432,0.000000,-0.367050,-0.330641,-0.271686,2.097036,1.019964,-0.395779,-0.439621,0.636039,...,0.000000,1.723172,0.101098,0.801063,1.039571,0.000000,0.000000,0.000000,-0.181589,0.000000
3,0.000000,0.000000,0.000000,-0.088963,0.000000,0.000000,0.484141,0.000000,-0.439621,0.910607,...,0.000000,-0.661305,-0.188482,0.000000,0.000000,0.000000,0.000000,0.000000,-0.139623,0.000000
4,0.000000,0.000000,0.000000,1.240269,0.000000,0.000000,-1.694872,0.000000,0.348021,0.224186,...,0.000000,0.603034,-0.157922,0.476344,0.680205,-1.333059,-0.317391,0.000000,-0.414731,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16755,0.000000,1.044231,0.000000,-0.572320,0.000000,0.290121,0.091204,0.000000,-0.200239,0.407232,...,0.000000,0.667703,-1.013160,0.000000,0.000000,0.000000,-0.361918,0.584864,-0.156720,0.000000
16756,0.000000,0.000000,0.000000,-0.290362,0.000000,-0.677870,-2.230694,0.000000,-0.331513,-0.599519,...,0.000000,-0.389591,-0.067515,0.000000,0.000000,0.000000,-0.364145,0.973702,0.429243,0.000000
16757,-0.260876,1.044231,-0.297857,-0.330641,-0.261407,0.000000,1.448622,-0.151648,0.116362,0.361470,...,0.000000,-1.128922,-0.353408,0.000000,0.000000,0.000000,0.000000,-0.443803,-0.682067,0.000000
16758,0.000000,0.000000,0.000000,-0.753579,0.000000,0.000000,-0.051682,0.000000,-0.219544,-0.788284,...,0.000000,2.021813,0.149653,0.000000,0.000000,0.000000,0.000000,0.782719,-0.633884,-0.303768


In [52]:
train_label = train_label.values.ravel()
valid_label = valid_label.values.ravel()

In [53]:
##训练第一层分类器
##Stacking建模------------不调参
from sklearn.model_selection import KFold
base_learners = get_models()
kf = KFold(n_splits=5,shuffle=False)
def get_oof(model,x_train,y_train,x_test):
    oof_train=np.zeros((x_train.shape[0],))     
    oof_test=np.zeros((x_test.shape[0],))       
    oof_test_skf=np.zeros((5,x_test.shape[0]))  
    for i,(train_index,test_index) in enumerate(kf.split(x_train)): 
        kf_x_train=x_train[train_index] 
        kf_y_train=y_train[train_index]              
        kf_x_test=x_train[test_index]                
        model=model.fit(kf_x_train,kf_y_train)
        oof_train[test_index]=model.predict(kf_x_test)       
        oof_test_skf[i,:]=model.predict(x_test)             
    oof_test[:]=oof_test_skf.mean(axis=0)      
    return oof_train,oof_test

number_models=len(base_learners)
xtrain_new=np.zeros((train_feature.shape[0],number_models))
xtest_new=np.zeros((valid_feature.shape[0],number_models))
for i, (name, m) in enumerate(base_learners.items()):
    xtrain_new[:,i],xtest_new[:,i]=get_oof(m,train_feature.values,train_label,valid_feature.values)

In [54]:
import catboost as cat
meta_learner_no_params = cat.CatBoostRegressor(iterations = 2000)
meta_learner_no_params.fit(xtrain_new, train_label)

##预测
pred = meta_learner_no_params.predict(valid_feature.values)

rmse = np.sqrt(mean_squared_error(valid_label,pred))
print(rmse)

Learning rate set to 0.036381
0:	learn: 0.2580043	total: 3.28ms	remaining: 6.56s
1:	learn: 0.2558430	total: 5.3ms	remaining: 5.29s
2:	learn: 0.2538703	total: 7.26ms	remaining: 4.83s
3:	learn: 0.2519768	total: 9.18ms	remaining: 4.58s
4:	learn: 0.2501705	total: 11.1ms	remaining: 4.42s
5:	learn: 0.2484364	total: 13ms	remaining: 4.33s
6:	learn: 0.2468678	total: 15.1ms	remaining: 4.3s
7:	learn: 0.2453851	total: 17ms	remaining: 4.24s
8:	learn: 0.2439408	total: 18.9ms	remaining: 4.18s
9:	learn: 0.2426330	total: 20.9ms	remaining: 4.16s
10:	learn: 0.2413672	total: 22.9ms	remaining: 4.14s
11:	learn: 0.2401600	total: 25.1ms	remaining: 4.16s
12:	learn: 0.2390220	total: 27.1ms	remaining: 4.13s
13:	learn: 0.2379978	total: 29ms	remaining: 4.12s
14:	learn: 0.2370593	total: 30.9ms	remaining: 4.08s
15:	learn: 0.2361138	total: 32.8ms	remaining: 4.07s
16:	learn: 0.2352518	total: 34.7ms	remaining: 4.05s
17:	learn: 0.2344328	total: 36.6ms	remaining: 4.04s
18:	learn: 0.2336759	total: 38.6ms	remaining: 4.03s


176:	learn: 0.2174178	total: 349ms	remaining: 3.59s
177:	learn: 0.2173830	total: 350ms	remaining: 3.59s
178:	learn: 0.2173656	total: 352ms	remaining: 3.58s
179:	learn: 0.2173269	total: 354ms	remaining: 3.58s
180:	learn: 0.2173126	total: 356ms	remaining: 3.58s
181:	learn: 0.2172935	total: 359ms	remaining: 3.58s
182:	learn: 0.2172765	total: 361ms	remaining: 3.58s
183:	learn: 0.2172507	total: 363ms	remaining: 3.59s
184:	learn: 0.2172074	total: 366ms	remaining: 3.59s
185:	learn: 0.2171856	total: 368ms	remaining: 3.58s
186:	learn: 0.2171508	total: 370ms	remaining: 3.58s
187:	learn: 0.2170648	total: 372ms	remaining: 3.58s
188:	learn: 0.2170266	total: 374ms	remaining: 3.58s
189:	learn: 0.2170141	total: 376ms	remaining: 3.58s
190:	learn: 0.2169870	total: 378ms	remaining: 3.58s
191:	learn: 0.2169708	total: 380ms	remaining: 3.57s
192:	learn: 0.2169224	total: 382ms	remaining: 3.57s
193:	learn: 0.2168401	total: 383ms	remaining: 3.57s
194:	learn: 0.2168061	total: 385ms	remaining: 3.57s
195:	learn: 

355:	learn: 0.2110624	total: 708ms	remaining: 3.27s
356:	learn: 0.2110042	total: 710ms	remaining: 3.27s
357:	learn: 0.2109817	total: 712ms	remaining: 3.27s
358:	learn: 0.2109324	total: 714ms	remaining: 3.27s
359:	learn: 0.2108885	total: 716ms	remaining: 3.26s
360:	learn: 0.2108660	total: 718ms	remaining: 3.26s
361:	learn: 0.2108483	total: 720ms	remaining: 3.26s
362:	learn: 0.2108069	total: 722ms	remaining: 3.26s
363:	learn: 0.2107455	total: 725ms	remaining: 3.26s
364:	learn: 0.2106876	total: 727ms	remaining: 3.25s
365:	learn: 0.2106664	total: 729ms	remaining: 3.25s
366:	learn: 0.2106300	total: 731ms	remaining: 3.25s
367:	learn: 0.2106159	total: 733ms	remaining: 3.25s
368:	learn: 0.2105729	total: 735ms	remaining: 3.25s
369:	learn: 0.2105384	total: 737ms	remaining: 3.25s
370:	learn: 0.2105164	total: 739ms	remaining: 3.24s
371:	learn: 0.2104997	total: 741ms	remaining: 3.24s
372:	learn: 0.2104685	total: 743ms	remaining: 3.24s
373:	learn: 0.2104310	total: 745ms	remaining: 3.24s
374:	learn: 

528:	learn: 0.2057086	total: 1.07s	remaining: 2.97s
529:	learn: 0.2056865	total: 1.07s	remaining: 2.96s
530:	learn: 0.2056627	total: 1.07s	remaining: 2.96s
531:	learn: 0.2056260	total: 1.07s	remaining: 2.96s
532:	learn: 0.2055777	total: 1.07s	remaining: 2.96s
533:	learn: 0.2055614	total: 1.08s	remaining: 2.96s
534:	learn: 0.2055267	total: 1.08s	remaining: 2.96s
535:	learn: 0.2054930	total: 1.08s	remaining: 2.96s
536:	learn: 0.2054819	total: 1.08s	remaining: 2.95s
537:	learn: 0.2054300	total: 1.09s	remaining: 2.96s
538:	learn: 0.2053871	total: 1.09s	remaining: 2.96s
539:	learn: 0.2053505	total: 1.09s	remaining: 2.96s
540:	learn: 0.2053153	total: 1.09s	remaining: 2.95s
541:	learn: 0.2052862	total: 1.1s	remaining: 2.95s
542:	learn: 0.2052324	total: 1.1s	remaining: 2.95s
543:	learn: 0.2051991	total: 1.1s	remaining: 2.95s
544:	learn: 0.2051757	total: 1.1s	remaining: 2.95s
545:	learn: 0.2051348	total: 1.1s	remaining: 2.94s
546:	learn: 0.2050864	total: 1.11s	remaining: 2.94s
547:	learn: 0.205

699:	learn: 0.2005593	total: 1.43s	remaining: 2.65s
700:	learn: 0.2005202	total: 1.43s	remaining: 2.65s
701:	learn: 0.2004890	total: 1.43s	remaining: 2.65s
702:	learn: 0.2004355	total: 1.43s	remaining: 2.65s
703:	learn: 0.2004026	total: 1.44s	remaining: 2.64s
704:	learn: 0.2003898	total: 1.44s	remaining: 2.64s
705:	learn: 0.2003726	total: 1.44s	remaining: 2.64s
706:	learn: 0.2003388	total: 1.44s	remaining: 2.64s
707:	learn: 0.2003037	total: 1.44s	remaining: 2.63s
708:	learn: 0.2002742	total: 1.45s	remaining: 2.63s
709:	learn: 0.2002385	total: 1.45s	remaining: 2.63s
710:	learn: 0.2002179	total: 1.45s	remaining: 2.63s
711:	learn: 0.2002011	total: 1.45s	remaining: 2.63s
712:	learn: 0.2001642	total: 1.45s	remaining: 2.62s
713:	learn: 0.2001285	total: 1.46s	remaining: 2.62s
714:	learn: 0.2000804	total: 1.46s	remaining: 2.62s
715:	learn: 0.2000690	total: 1.46s	remaining: 2.62s
716:	learn: 0.2000568	total: 1.46s	remaining: 2.62s
717:	learn: 0.2000437	total: 1.46s	remaining: 2.62s
718:	learn: 

...

1230:	learn: 0.1890100	total: 2.5s	remaining: 1.56s
1231:	learn: 0.1889795	total: 2.51s	remaining: 1.56s
1232:	learn: 0.1889711	total: 2.51s	remaining: 1.56s
1233:	learn: 0.1889467	total: 2.51s	remaining: 1.56s
1234:	learn: 0.1889406	total: 2.51s	remaining: 1.56s
1235:	learn: 0.1889057	total: 2.52s	remaining: 1.55s
1236:	learn: 0.1888974	total: 2.52s	remaining: 1.55s
1237:	learn: 0.1888749	total: 2.52s	remaining: 1.55s
1238:	learn: 0.1888607	total: 2.52s	remaining: 1.55s
1239:	learn: 0.1888555	total: 2.52s	remaining: 1.55s
1240:	learn: 0.1888277	total: 2.52s	remaining: 1.54s
1241:	learn: 0.1888217	total: 2.53s	remaining: 1.54s
1242:	learn: 0.1888042	total: 2.53s	remaining: 1.54s
1243:	learn: 0.1887871	total: 2.53s	remaining: 1.54s
1244:	learn: 0.1887707	total: 2.53s	remaining: 1.53s
1245:	learn: 0.1887565	total: 2.53s	remaining: 1.53s
1246:	learn: 0.1887532	total: 2.54s	remaining: 1.53s
1247:	learn: 0.1887484	total: 2.54s	remaining: 1.53s
1248:	learn: 0.1887298	total: 2.54s	remaining: 

1401:	learn: 0.1863236	total: 2.86s	remaining: 1.22s
1402:	learn: 0.1863020	total: 2.87s	remaining: 1.22s
1403:	learn: 0.1862820	total: 2.87s	remaining: 1.22s
1404:	learn: 0.1862616	total: 2.87s	remaining: 1.22s
1405:	learn: 0.1862299	total: 2.87s	remaining: 1.21s
1406:	learn: 0.1862192	total: 2.87s	remaining: 1.21s
1407:	learn: 0.1862088	total: 2.88s	remaining: 1.21s
1408:	learn: 0.1861952	total: 2.88s	remaining: 1.21s
1409:	learn: 0.1861786	total: 2.88s	remaining: 1.21s
1410:	learn: 0.1861655	total: 2.88s	remaining: 1.2s
1411:	learn: 0.1861601	total: 2.88s	remaining: 1.2s
1412:	learn: 0.1861549	total: 2.89s	remaining: 1.2s
1413:	learn: 0.1861466	total: 2.89s	remaining: 1.2s
1414:	learn: 0.1861326	total: 2.89s	remaining: 1.2s
1415:	learn: 0.1861223	total: 2.89s	remaining: 1.19s
1416:	learn: 0.1860981	total: 2.9s	remaining: 1.19s
1417:	learn: 0.1860833	total: 2.9s	remaining: 1.19s
1418:	learn: 0.1860656	total: 2.9s	remaining: 1.19s
1419:	learn: 0.1860620	total: 2.9s	remaining: 1.19s
14

1570:	learn: 0.1835160	total: 3.22s	remaining: 880ms
1571:	learn: 0.1835051	total: 3.22s	remaining: 878ms
1572:	learn: 0.1835035	total: 3.23s	remaining: 876ms
1573:	learn: 0.1834791	total: 3.23s	remaining: 874ms
1574:	learn: 0.1834723	total: 3.23s	remaining: 872ms
1575:	learn: 0.1834625	total: 3.23s	remaining: 870ms
1576:	learn: 0.1834491	total: 3.23s	remaining: 868ms
1577:	learn: 0.1834390	total: 3.24s	remaining: 866ms
1578:	learn: 0.1834324	total: 3.24s	remaining: 864ms
1579:	learn: 0.1834199	total: 3.24s	remaining: 862ms
1580:	learn: 0.1833997	total: 3.24s	remaining: 860ms
1581:	learn: 0.1833705	total: 3.25s	remaining: 858ms
1582:	learn: 0.1833478	total: 3.25s	remaining: 856ms
1583:	learn: 0.1833276	total: 3.25s	remaining: 854ms
1584:	learn: 0.1832923	total: 3.25s	remaining: 852ms
1585:	learn: 0.1832787	total: 3.25s	remaining: 849ms
1586:	learn: 0.1832539	total: 3.26s	remaining: 848ms
1587:	learn: 0.1832324	total: 3.26s	remaining: 845ms
1588:	learn: 0.1832113	total: 3.26s	remaining:

1744:	learn: 0.1807057	total: 3.58s	remaining: 523ms
1745:	learn: 0.1806952	total: 3.58s	remaining: 521ms
1746:	learn: 0.1806832	total: 3.58s	remaining: 519ms
1747:	learn: 0.1806696	total: 3.58s	remaining: 517ms
1748:	learn: 0.1806559	total: 3.59s	remaining: 515ms
1749:	learn: 0.1806444	total: 3.59s	remaining: 513ms
1750:	learn: 0.1806197	total: 3.59s	remaining: 511ms
1751:	learn: 0.1805964	total: 3.6s	remaining: 509ms
1752:	learn: 0.1805841	total: 3.6s	remaining: 507ms
1753:	learn: 0.1805718	total: 3.6s	remaining: 505ms
1754:	learn: 0.1805528	total: 3.6s	remaining: 503ms
1755:	learn: 0.1805392	total: 3.6s	remaining: 501ms
1756:	learn: 0.1805111	total: 3.61s	remaining: 499ms
1757:	learn: 0.1804951	total: 3.61s	remaining: 497ms
1758:	learn: 0.1804823	total: 3.61s	remaining: 495ms
1759:	learn: 0.1804588	total: 3.61s	remaining: 493ms
1760:	learn: 0.1804478	total: 3.62s	remaining: 491ms
1761:	learn: 0.1804381	total: 3.62s	remaining: 489ms
1762:	learn: 0.1804187	total: 3.62s	remaining: 487m

1923:	learn: 0.1780035	total: 3.94s	remaining: 156ms
1924:	learn: 0.1779944	total: 3.94s	remaining: 153ms
1925:	learn: 0.1779821	total: 3.94s	remaining: 151ms
1926:	learn: 0.1779779	total: 3.94s	remaining: 149ms
1927:	learn: 0.1779549	total: 3.94s	remaining: 147ms
1928:	learn: 0.1779463	total: 3.95s	remaining: 145ms
1929:	learn: 0.1779258	total: 3.95s	remaining: 143ms
1930:	learn: 0.1779097	total: 3.95s	remaining: 141ms
1931:	learn: 0.1779016	total: 3.95s	remaining: 139ms
1932:	learn: 0.1778879	total: 3.96s	remaining: 137ms
1933:	learn: 0.1778770	total: 3.96s	remaining: 135ms
1934:	learn: 0.1778575	total: 3.96s	remaining: 133ms
1935:	learn: 0.1778399	total: 3.96s	remaining: 131ms
1936:	learn: 0.1778299	total: 3.96s	remaining: 129ms
1937:	learn: 0.1778189	total: 3.96s	remaining: 127ms
1938:	learn: 0.1778134	total: 3.97s	remaining: 125ms
1939:	learn: 0.1777938	total: 3.97s	remaining: 123ms
1940:	learn: 0.1777768	total: 3.97s	remaining: 121ms
1941:	learn: 0.1777547	total: 3.97s	remaining:

In [55]:
##模型调参
from hyperopt import hp, tpe, fmin,STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss
from sklearn.model_selection import cross_val_score,KFold

##定义第二层的要优化的参数函数
def hyperopt_objective(params):
    model=cat.CatBoostRegressor(iterations=int(params['iterations'])
              ,max_depth=int(params['max_depth'])
              ,l2_leaf_reg=params['l2_leaf_reg']
              ,learning_rate=params['learning_rate']
              )
    
    cv=KFold(n_splits=5,shuffle=True,random_state=3612)
    validate_loss=abs(cross_val_score(model,train_feature,train_label
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')).mean()
    
    return validate_loss

 ##生成参数范围
iterations_range = range(1500,2500,50)
depth_range = range(1,10,1)
l2_leaf_reg_range = range(2,8,1)

param_grid_simple={'iterations':hp.choice('iterations',iterations_range)
                   ,'max_depth':hp.choice('max_depth',depth_range)
                   ,'l2_leaf_reg':hp.choice('l2_leaf_reg',l2_leaf_reg_range)
                   ,'learning_rate':hp.quniform('learning_rate',0.001,0.5,0.001)
                  }

def param_hyperopt(max_evals=100):
    # 记录迭代过程
    trials=Trials()
    
    # 提前停止
    early_stop_fn=no_progress_loss(50) 

    params_best=fmin(hyperopt_objective # 设定目标函数
                     ,space=param_grid_simple 
                     ,algo=tpe.suggest 
                     ,max_evals=max_evals # 设定迭代次数
                     ,trials=trials 
                     ,early_stop_fn=early_stop_fn # 控制提前停止
                    )
    
    print('best parmas:',params_best)
    return params_best,trials
print('Stacking贝叶斯优化开始------------')
params_best,trials = param_hyperopt(max_evals=50)
print('Stacking贝叶斯优化结束------------')

Stacking贝叶斯优化开始------------
  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

job exception: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.



  0%|                                                                           | 0/50 [00:01<?, ?trial/s, best loss=?]


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
meta_learner = cat.CatBoostRegressor(max_depth = depth_range[params_best['max_depth']],
                                      iterations = iterations_range[params_best['iterations']],
                                      l2_leaf_reg = l2_leaf_reg_range[params_best['l2_leaf_reg']])
meta_learner.fit(train_feature, train_label)



##预测
pred_param = meta_learner.predict(valid_feature.values)

In [ ]:
def feature_importance_catboost(model):
    result=pd.DataFrame(model.get_feature_importance(),index=model.feature_names_,columns=['FeatureImportance'])
    return result.sort_values('FeatureImportance',ascending=False)
df=feature_importance_catboost(meta_learner)
newco=df.drop(df.index[55:]).index
df.drop(df.index[55:])
#newco=df.drop(df.index[51:]).index

In [ ]:
##模型调参
from hyperopt import hp, tpe, fmin,STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss
from sklearn.model_selection import cross_val_score,KFold

##定义第二层的要优化的参数函数
def hyperopt_objective(params):
    model=cat.CatBoostRegressor(iterations=int(params['iterations'])
              ,max_depth=int(params['max_depth'])
              ,l2_leaf_reg=params['l2_leaf_reg']
              ,learning_rate=params['learning_rate']
              )
    
    cv=KFold(n_splits=5,shuffle=True,random_state=7)
    validate_loss=abs(cross_val_score(model,trainnewpd,train_label
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')).mean()
    
    return validate_loss

 ##生成参数范围
iterations_range = range(1500,2500,50)
depth_range = range(1,10,1)
l2_leaf_reg_range = range(2,8,1)

param_grid_simple={'iterations':hp.choice('iterations',iterations_range)
                   ,'max_depth':hp.choice('max_depth',depth_range)
                   ,'l2_leaf_reg':hp.choice('l2_leaf_reg',l2_leaf_reg_range)
                   ,'learning_rate':hp.quniform('learning_rate',0.001,0.5,0.001)
                  }

def param_hyperopt(max_evals=100):
    # 记录迭代过程
    trials=Trials()
    
    # 提前停止
    early_stop_fn=no_progress_loss(50) 

    params_best=fmin(hyperopt_objective
                     ,space=param_grid_simple 
                     ,algo=tpe.suggest 
                     ,max_evals=max_evals 
                     ,trials=trials 
                     ,early_stop_fn=early_stop_fn
                    )
    
    print('best parmas:',params_best)
    return params_best,trials
print('Stacking贝叶斯优化开始------------')
params_best,trials = param_hyperopt(max_evals=50)
print('Stacking贝叶斯优化结束------------')

In [ ]:
trainnew=[]
validnew=[]
testnew=[]
for i in newco:
    trainnew.append(train_feature[i])
    validnew.append(valid_feature[i])
    testnew.append(test_1[i])
trainnewpd=pd.DataFrame(trainnew).T
validnewpd=pd.DataFrame(validnew).T
testnewpd=pd.DataFrame(testnew).T

In [ ]:
meta_learner.fit(trainnewpd, train_label)

pred_param = meta_learner.predict(validnewpd)

In [ ]:
print(np.sqrt(mean_squared_error(valid_label.tolist(),pred_param)))

In [ ]:
pred = meta_learner.predict(test_1)
pd.DataFrame(pred).to_csv('night.csv',index=False)